# CCPA vs. ERA5 data analysis

In [1]:
import os
import sys
import time
import h5py

import numpy as np
import pandas as pd
import xarray as xr
from glob import glob
from scipy.stats import skew, kurtosis

from datetime import datetime, timedelta

In [2]:
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/')
sys.path.insert(0, '/glade/u/home/ksha/GAN_proj/libs/')

from namelist import *
import data_utils as du

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

## Data preparation

**CONUS Land mask for CCPA**

In [4]:
ds_static = xr.open_dataset(save_dir+'CCPA_domain.hdf')

ds_static = ds_static.rename({
    'phony_dim_0': 'latitude',
    'phony_dim_1': 'longitude'
})

ds_static = ds_static.assign_coords({
    'latitude': ds_static['lat_CCPA'].values[:, 0],
    'longitude': ds_static['lon_CCPA'].values[0, :]
})

CCPA_y = ds_static['latitude'].values
CCPA_x = ds_static['longitude'].values
CCPA_x = (CCPA_x + 360) % 360
ds_static = ds_static.assign_coords({'longitude': CCPA_x})

**CONUS Land mask for ERA5**

In [5]:
ERA5_fn = '/glade/derecho/scratch/ksha/CREDIT_data/ERA5_plevel_base/accum/ERA5_plevel_6h_accum_{}.zarr'
ds_ERA5_example = xr.open_zarr(ERA5_fn.format(1979))

In [6]:
ERA5_y = ds_ERA5_example['latitude'].values
ERA5_x = ds_ERA5_example['longitude'].values
ERA5_lon, ERA5_lat = np.meshgrid(ERA5_x, ERA5_y)

**Prepare the CONUS mask using nearest neighbour (run once only)**

In [7]:
# import xesmf as xe

# ds_out = xr.Dataset(
#     {
#         'latitude': (['latitude'], ERA5_y[120:280]),
#         'longitude': (['longitude'], ERA5_x[720:])
#     }
# )

# regridder = xe.Regridder(ds_static, ds_out, 'bilinear')
# land_mask_ERA5 = regridder(ds_static['land_mask_CCPA'])
# land_mask_ERA5 = xr.where(land_mask_ERA5 >= 0.5, 1, 0)

# ds_land_mask = land_mask_ERA5.to_dataset(name='land_mask_ERA5')
# ds_land_mask.to_netcdf(save_dir+'CCPA_ERA5_domain.nc', format='NETCDF4')

ds_land_mask = xr.open_dataset(save_dir+'CCPA_ERA5_domain.nc')

## CCPA data analysis routine

In [8]:
def skew_func(data, axis):
    return skew(data, axis=axis, nan_policy='omit')

def kurtosis_func(data, axis):
    return kurtosis(data, axis=axis, nan_policy='omit')

### Data gathering

In [9]:
fn_CCPA = camp_dir+'CCPA/CCPA_y{}.hdf'

# define the range of years (20-yr)
years = np.arange(2002, 2022)

# list to hold datasets for each year
ds_list = []

for year in years:
    # open HDF5
    ds_CCPA = xr.open_dataset(fn_CCPA.format(year))
    
    # rename HDF5 coords to xarray coords
    ds_CCPA = ds_CCPA.rename({
        'phony_dim_0': 'day',
        'phony_dim_1': 'hour',
        'phony_dim_2': 'latitude',
        'phony_dim_3': 'longitude'
    })
    
    ds_CCPA = ds_CCPA.assign_coords({
        'day': ds_CCPA['day'],
        'hour': ds_CCPA['hour'],
        'latitude': ds_static['latitude'].values,
        'longitude': ds_static['longitude'].values
    })
    
    # mask non-US grid as NaNs
    ds_CCPA = ds_CCPA.where(ds_static['land_mask_CCPA'] != 0)
    
    # convert day / hour coords to time
    ds_CCPA_stack = ds_CCPA.stack(time=('day', 'hour'))
    ds_CCPA_stack = ds_CCPA_stack.reset_index('time')
    start_date = pd.Timestamp(f'{year}-01-01')
    actual_hours = ds_CCPA_stack['hour'] * 6
    time_values = start_date + pd.to_timedelta(ds_CCPA_stack['day'], unit='D') + pd.to_timedelta(actual_hours, unit='H')
    ds_CCPA_stack = ds_CCPA_stack.assign_coords(time=('time', time_values))
    ds_CCPA_stack = ds_CCPA_stack.drop_vars(['day', 'hour'])
    
    # re-order dims
    ds_CCPA_stack['CCPA'] = ds_CCPA_stack['CCPA'].transpose('time', 'latitude', 'longitude')
    
    ds_list.append(ds_CCPA_stack)

ds_CCPA_FULL = xr.concat(ds_list, dim='time')

In [10]:
ds_CCPA_FULL

<xarray.Dataset>
Dimensions:    (time: 29220, latitude: 224, longitude: 464)
Coordinates:
  * latitude   (latitude) float64 25.06 25.19 25.31 25.44 ... 52.69 52.81 52.94
  * longitude  (longitude) float64 235.1 235.2 235.3 235.4 ... 292.7 292.8 292.9
  * time       (time) datetime64[ns] 2002-01-01 ... 2021-12-31T18:00:00
Data variables:
    CCPA       (time, latitude, longitude) float64 nan nan nan ... nan nan nan

### Time series analysis

In [ ]:
CCPA_mean_series = ds_CCPA_FULL.mean(dim=['latitude', 'longitude'], skipna=True)
CCPA_std_series = ds_CCPA_FULL.std(dim=['latitude', 'longitude'], skipna=True)

data_var = ds_CCPA_FULL['CCPA']

q_vals = [0.90, 0.95, 0.99]

quantiles = data_var.quantile(q=q_vals, dim=['latitude', 'longitude'], skipna=True)

# Extract individual percentiles
quantile_90 = quantiles.sel(quantile=0.90)
quantile_95 = quantiles.sel(quantile=0.95)
quantile_99 = quantiles.sel(quantile=0.99)

ds_CCPA_series = xr.Dataset({
    'mean': CCPA_mean_series['CCPA'],
    'std': CCPA_std_series['CCPA'],
    'q90': quantile_90.drop_vars('quantile'),
    'q95': quantile_95.drop_vars('quantile'),
    'q99': quantile_99.drop_vars('quantile'),
})

# ds_CCPA_series.to_netcdf(save_dir+'CCPA_2002_2022_series.nc', format='NETCDF4')

### Spatial distribution analysis

In [19]:
CCPA_mean = ds_CCPA_FULL.mean(dim='time', skipna=True)
CCPA_std = ds_CCPA_FULL.std(dim='time', skipna=True)

data_var = ds_CCPA_FULL['CCPA']

CCPA_skew = xr.apply_ufunc(
    skew_func,
    data_var,
    input_core_dims=[['time']],
    kwargs={'axis': -1},
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float],
)

# Compute kurtosis over 'time' dimension
CCPA_kurt = xr.apply_ufunc(
    kurtosis_func,
    data_var,
    input_core_dims=[['time']],
    kwargs={'axis': -1},
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float],
)

CCPA_kurt_np = CCPA_kurt.values

q_vals = [0.90, 0.95, 0.99]

quantiles = data_var.quantile(q=q_vals, dim='time', skipna=True)

# Extract individual percentiles
quantile_90 = quantiles.sel(quantile=0.90)
quantile_95 = quantiles.sel(quantile=0.95)
quantile_99 = quantiles.sel(quantile=0.99)

ds_CCPA_analysis = xr.Dataset({
    'mean': CCPA_mean['CCPA'],
    'std': CCPA_std['CCPA'],
    'skew': CCPA_skew,
    'kurt': CCPA_kurt,
    'q90': quantile_90.drop_vars('quantile'),
    'q95': quantile_95.drop_vars('quantile'),
    'q99': quantile_99.drop_vars('quantile'),
})

#ds_CCPA_analysis.to_netcdf(save_dir+'CCPA_2002_2022_analysis.nc', format='NETCDF4')

## ERA5 (CONUS domain) routine

In [11]:
years = np.arange(2002, 2022)

# list to hold datasets for each year
ds_list_ERA5 = []

for year in years:
    ds_ERA5 = xr.open_zarr(ERA5_fn.format(year))

    da_precip = ds_ERA5['total_precipitation']
    da_precip = da_precip.isel(latitude=slice(120, 280), longitude=slice(720, None))
    da_precip = da_precip.where(ds_land_mask['land_mask_ERA5'] != 0)
    ds_precip = da_precip.to_dataset(name='ERA5')
    ds_list_ERA5.append(ds_precip)

ds_ERA5_FULL = xr.concat(ds_list_ERA5, dim='time')

In [12]:
# rechunk time dim
ds_ERA5_FULL = ds_ERA5_FULL.chunk(dict(time=-1))

In [77]:
ds_ERA5_FULL

<xarray.Dataset>
Dimensions:    (latitude: 160, longitude: 720, time: 29220)
Coordinates:
  * latitude   (latitude) float32 60.0 59.75 59.5 59.25 ... 20.75 20.5 20.25
  * longitude  (longitude) float32 180.0 180.2 180.5 180.8 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 2002-01-01 ... 2021-12-31T18:00:00
Data variables:
    ERA5       (time, latitude, longitude) float32 dask.array<chunksize=(29220, 160, 720), meta=np.ndarray>

### Time series analysis

In [17]:
ERA5_mean_series = ds_ERA5_FULL.mean(dim=['latitude', 'longitude'], skipna=True)
ERA5_std_series = ds_ERA5_FULL.std(dim=['latitude', 'longitude'], skipna=True)

data_var = ds_ERA5_FULL['ERA5']

q_vals = [0.90, 0.95, 0.99]

quantiles = data_var.quantile(q=q_vals, dim=['latitude', 'longitude'], skipna=True)

# Extract individual percentiles
quantile_90 = quantiles.sel(quantile=0.90)
quantile_95 = quantiles.sel(quantile=0.95)
quantile_99 = quantiles.sel(quantile=0.99)

ds_ERA5_series = xr.Dataset({
    'mean': ERA5_mean_series['ERA5'],
    'std': ERA5_std_series['ERA5'],
    'q90': quantile_90.drop_vars('quantile'),
    'q95': quantile_95.drop_vars('quantile'),
    'q99': quantile_99.drop_vars('quantile'),
})

# ds_ERA5_series.to_netcdf(save_dir+'ERA5_2002_2022_series.nc', format='NETCDF4')

### Spatial distribution analysis

In [ ]:
ERA5_mean = ds_ERA5_FULL.mean(dim='time', skipna=True)
ERA5_std = ds_ERA5_FULL.std(dim='time', skipna=True)

data_var = ds_ERA5_FULL['ERA5']

ERA5_skew = xr.apply_ufunc(
    skew_func,
    data_var,
    input_core_dims=[['time']],
    kwargs={'axis': -1},
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float],
)

ERA5_kurt = xr.apply_ufunc(
    kurtosis_func,
    data_var,
    input_core_dims=[['time']],
    kwargs={'axis': -1},
    vectorize=True,
    dask='parallelized',
    output_dtypes=[float],
)

ERA5_kurt_np = ERA5_kurt.values

q_vals = [0.90, 0.95, 0.99]

quantiles_ERA5 = data_var.quantile(q=q_vals, dim='time', skipna=True)

# Extract individual percentiles
quantile_ERA5_90 = quantiles_ERA5.sel(quantile=0.90)
quantile_ERA5_95 = quantiles_ERA5.sel(quantile=0.95)
quantile_ERA5_99 = quantiles_ERA5.sel(quantile=0.99)

ds_ERA5_analysis = xr.Dataset({
    'mean': ERA5_mean['ERA5'],
    'std': ERA5_std['ERA5'],
    'skew': ERA5_skew,
    'kurt': ERA5_kurt,
    'q90': quantile_ERA5_90.drop_vars('quantile'),
    'q95': quantile_ERA5_95.drop_vars('quantile'),
    'q99': quantile_ERA5_99.drop_vars('quantile'),
})

# ds_ERA5_analysis.to_netcdf(save_dir+'ERA5_2002_2022_CONUS_analysis.nc', format='NETCDF4')

/glade/derecho/scratch/ksha/tmp/ipykernel_64387/319367502.py:5: SmallSampleWarning: After omitting NaNs, one or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  return kurtosis(data, axis=axis, nan_policy='omit')
